# Generador de música
### Aquest projecte serveix per generar notes musicals seguint una estructura segons el gènere musical seleccionat

Importem els paquets necessaris

In [ ]:
import os
from pathlib import Path

def obtener_track_id_desde_path(midi_path):
    path = Path(midi_path)
    return path.parent.name if len(path.parts) >= 2 else None

print(obtener_track_id_desde_path("lmd_matched/A/A/A/TRAAAGR128F425B14B/1d9d16a9da90c090809c153754823c2b.mid"))

In [ ]:
import os
import json

LMD_MATCHED_DIR = 'lmd_matched'
MSD_METADATA_DIR = 'lastfm_train'

# Géneros que te interesan
COMMON_GENRES = {
    'rock', 'pop', 'jazz', 'blues', 'hip hop', 'rap', 'electronic',
    'dance', 'classical', 'metal', 'punk', 'country', 'folk',
    'reggae', 'r&b', 'soul', 'disco', 'house', 'techno', 'funk',
    'alternative', 'indie', 'grunge', 'trance', 'synthpop', 'electropop'
}

# Normaliza el texto del género
def normalize_genre(genre):
    return genre.lower().strip()

# Carga los metadatos y extrae el género válido
def extract_genre(mbid):
    metadata_path = os.path.join(MSD_METADATA_DIR, mbid + '.json')
    if not os.path.exists(metadata_path):
        return None

    with open(metadata_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        print(data)

    tags = data.get('tags', [])
    for tag_entry in tags:
        if isinstance(tag_entry, list) and len(tag_entry) >= 1:
            tag = tag_entry[0].lower().strip()
            if tag in COMMON_GENRES:
                return tag
    return None

# Asociación final: solo mbid, midi_path y género
midi_with_genres = []

for root, _, files in os.walk(LMD_MATCHED_DIR):
    for file in files:
        if file.lower().endswith('.mid') or file.lower().endswith('.midi'):
            mbid = os.path.basename(root)
            midi_path = os.path.join(root, file)
            genre = extract_genre(mbid)

            if genre:
                midi_with_genres.append({
                    'mbid': mbid,
                    'midi_path': midi_path,
                    'genre': genre
                })

# Guardamos el resultado
with open('lmd_genre_filtered.json', 'w') as out_f:
    json.dump(midi_with_genres, out_f, indent=2)

print(f"Total MIDI con género válido: {len(midi_with_genres)}")


In [ ]:
import pretty_midi

midi = pretty_midi.PrettyMIDI("Midi Dataset Clean/ARMSTRONG_LOUIS/What_a_Wonderful_World.mid")

In [ ]:
from music21 import converter, chord
score = converter.parse("Midi Dataset Clean/ARMSTRONG_LOUIS/What_a_Wonderful_World.mid")
chords = score.chordify()

print(chords)

In [ ]:
import os
import json
import pretty_midi

def extract_chords_from_midi(midi_path):
    try:
        midi_data = pretty_midi.PrettyMIDI(midi_path)
        chords = set()

        for instrument in midi_data.instruments:
            if instrument.is_drum:
                continue
            notes = instrument.notes
            if len(notes) < 3:
                continue

            # Agrupar por tiempo cercano para detectar acordes
            notes.sort(key=lambda note: note.start)
            current_chord = []
            last_start = None
            for note in notes:
                if last_start is None or abs(note.start - last_start) < 0.05:
                    current_chord.append(note.pitch)
                    last_start = note.start
                else:
                    if len(current_chord) >= 3:
                        chord = tuple(sorted(current_chord))
                        chords.add(chord)
                    current_chord = [note.pitch]
                    last_start = note.start
            if len(current_chord) >= 3:
                chord = tuple(sorted(current_chord))
                chords.add(chord)

        return [list(chord) for chord in chords]
    except Exception as e:
        print(f"Error en {midi_path}: {e}")
        return []

# Ruta al directorio raíz
root_dir = "Midi Dataset Clean"
output = {}

for dirpath, dirnames, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.lower().endswith((".mid", ".midi")):
            midi_path = os.path.join(dirpath, filename)
            chords = extract_chords_from_midi(midi_path)
            if chords:
                output[os.path.relpath(midi_path, root_dir)] = chords

# Guardar a un JSON
with open("midi_chords.json", "w") as f:
    json.dump(output, f, indent=2)


In [ ]:
from music21 import chord, pitch

def midi_notes_to_chord_name(notes):
    try:
        c = chord.Chord(notes)
        return c.figure  # Ej: "C", "Am", "G7"
    except Exception:
        return None


In [ ]:
with open("midi_chords.json", "r") as f:
    data = json.load(f)

sequences = []
for midi_path, chords in data.items():
    sequence = []
    for chord_notes in chords:
        name = midi_notes_to_chord_name(chord_notes)
        if name:
            sequence.append(name)
    if sequence:
        sequences.append(sequence)


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

tokenizer = Tokenizer(filters='', lower=False)
tokenizer.fit_on_texts(sequences)
sequences_int = tokenizer.texts_to_sequences(sequences)

max_len = max(len(seq) for seq in sequences_int)
padded_sequences = pad_sequences(sequences_int, maxlen=max_len, padding='post')


In [ ]:
estructura = ["estrofa", "estrofa", "puente", "estribillo"] * 2


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64),
    LSTM(128, return_sequences=True),
    LSTM(128),
    Dense(len(tokenizer.word_index)+1, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')


In [ ]:
X = padded_sequences[:, :-1]
y = padded_sequences[:, 1:]

model.fit(X, y, epochs=50, batch_size=32)
